# BPE tokenizer

El tokenizador `BPE` (Byte Pair Encoding - codificación de pares de bytes) es un algoritmo de compresión de datos que se utiliza para crear un vocabulario de subpalabras a partir de un corpus de texto. Este algoritmo se basa en la frecuencia de los pares de bytes en el texto. Se popularizó porque fue utilizado como tokenizador por LLMs como GPT, GPT-2, RoBERTa, BART y DeBERTa

## Algoritmo de entrenamiento

Supongamos que tenemos un corpus de texto que solo contiene las siguientes palabras `hug, pug, pun, bun y hugs`, el primer paso consiste en crear un vocabulario con todos los caracteres presentes en el corpus, en nuestro caso será `b, g, h, n, p, s, u`

In [22]:
corpus_words = ["hug", "pug", "pun", "bun", "hugs"]

# Concatenate all the words in the corpus
initial_corpus_tokens = ""
for word in corpus_words:
    initial_corpus_tokens += word

# Convert the concatenated string to a set of tokens to get unique tokens
initial_corpus_tokens = set(initial_corpus_tokens)

print(f"Corpus words: {corpus_words}")
print(f"Initial corpus tokens: {initial_corpus_tokens}")
print(f"Number of initial corpus tokens: {len(initial_corpus_tokens)}")

Corpus words: ['hug', 'pug', 'pun', 'bun', 'hugs']
Initial corpus tokens: {'p', 'n', 'u', 's', 'h', 'g', 'b'}
Number of initial corpus tokens: 7


Ahora supongamos que este es nuestro corpus de frases, es un corpus inventado, no tiene sentido

In [23]:
corpus = [
    "hug hug hug pun pun bun hugs",
    "hug hug pug pug pun pun hugs",
    "hug hug pug pug pun pun pun pun hugs",
    "pug pun pun pun bun hugs",
    "hug hug hug pun bun bun hugs",
]

Vamos a contar el número de veces que aparece cada palabra en el corpus, para comprobar que lo que habíamos puesto antes está bien

In [24]:
num_hug = 0
num_pug = 0
num_pun = 0
num_bun = 0
num_hugs = 0

for sentence in corpus:
    words = sentence.split(" ")
    for word in words:
        if word == "hug":
            num_hug += 1
        elif word == "pug":
            num_pug += 1
        elif word == "pun":
            num_pun += 1
        elif word == "bun":
            num_bun += 1
        elif word == "hugs":
            num_hugs += 1

print(f"Number of hug: {num_hug}")
print(f"Number of pug: {num_pug}")
print(f"Number of pun: {num_pun}")
print(f"Number of bun: {num_bun}")
print(f"Number of hugs: {num_hugs}")

Number of hug: 10
Number of pug: 5
Number of pun: 12
Number of bun: 4
Number of hugs: 5


Todo lo que habíamos contado está bien, podemos seguir

Vamos a crear un diccionario con los tokens de cada palabra y el número de veces que aparece en el corpus

In [25]:
dict_tokens_by_word_appearance = {
    "hug":
        {
            "count": num_hug,
            "tokens": [character for character in "hug"],
        },
    "pug":
        {
            "count": num_pug,
            "tokens": [character for character in "pug"],
        },
    "pun":
        {
            "count": num_pun,
            "tokens": [character for character in "pun"],
        },
    "bun":
        {
            "count": num_bun,
            "tokens": [character for character in "bun"],
        },
    "hugs":
        {
            "count": num_hugs,
            "tokens": [character for character in "hugs"],
        },
}

dict_tokens_by_word_appearance

{'hug': {'count': 10, 'tokens': ['h', 'u', 'g']},
 'pug': {'count': 5, 'tokens': ['p', 'u', 'g']},
 'pun': {'count': 12, 'tokens': ['p', 'u', 'n']},
 'bun': {'count': 4, 'tokens': ['b', 'u', 'n']},
 'hugs': {'count': 5, 'tokens': ['h', 'u', 'g', 's']}}

Ahora vamos a buscar el par de tokens consecutivos que más veces aparece en el diccionario

In [26]:
dict_keys = dict_tokens_by_word_appearance.keys()

list_consecutive_tokens = []
for i, key in enumerate(dict_keys):
    # Get the tokens of the word
    number_of_toneks_of_word = len(dict_tokens_by_word_appearance[key]["tokens"])

    # Get consecituve tokens
    for j in range(number_of_toneks_of_word-1):
        # Get consecutive tokens
        consecutive_tokens = dict_tokens_by_word_appearance[key]["tokens"][j] + dict_tokens_by_word_appearance[key]["tokens"][j+1]
        # Append the consecutive tokens to the list the number of times the word appears
        for _ in range(dict_tokens_by_word_appearance[key]["count"]):
            list_consecutive_tokens.append(consecutive_tokens)
# Print the list of consecutive tokens
print(f"List of consecutive tokens: {list_consecutive_tokens}")

# Get consecutive tokens with maximum frequency
dict_consecutive_tokens = {}
for token in list_consecutive_tokens:
    # Check if the token is already in the dictionary
    if token in dict_consecutive_tokens:
        # Increment the count of the token
        dict_consecutive_tokens[token] += 1
    
    # If the token is not in the dictionary
    else:
        # Add the token to the dictionary
        dict_consecutive_tokens[token] = 1
# Print the dictionary of consecutive tokens
print(f"Dictionary of consecutive tokens: {dict_consecutive_tokens}")

# Get the consecutive token with maximum frequency
max_consecutive_token = None
while True:
    # Get the token with maximum frequency
    consecutive_token = max(dict_consecutive_tokens, key=dict_consecutive_tokens.get)

    # Check if the token is already in the list of tokens
    if consecutive_token in initial_corpus_tokens:
        # Remove token from the dictionary
        dict_consecutive_tokens.pop(consecutive_token)

    # If the token is not in the list of tokens
    else:
        # Assign the token to the max_consecutive_token
        max_consecutive_token = consecutive_token
        break

# Print the consecutive token with maximum frequency
print(f"Consecutive token with maximum frequency: {max_consecutive_token}")

List of consecutive tokens: ['hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'hu', 'ug', 'ug', 'ug', 'ug', 'ug', 'ug', 'ug', 'ug', 'ug', 'ug', 'pu', 'pu', 'pu', 'pu', 'pu', 'ug', 'ug', 'ug', 'ug', 'ug', 'pu', 'pu', 'pu', 'pu', 'pu', 'pu', 'pu', 'pu', 'pu', 'pu', 'pu', 'pu', 'un', 'un', 'un', 'un', 'un', 'un', 'un', 'un', 'un', 'un', 'un', 'un', 'bu', 'bu', 'bu', 'bu', 'un', 'un', 'un', 'un', 'hu', 'hu', 'hu', 'hu', 'hu', 'ug', 'ug', 'ug', 'ug', 'ug', 'gs', 'gs', 'gs', 'gs', 'gs']
Dictionary of consecutive tokens: {'hu': 15, 'ug': 20, 'pu': 17, 'un': 16, 'bu': 4, 'gs': 5}
Consecutive token with maximum frequency: ug


Hemos obtenido el par de tokens que más veces aparece. Vamos a encapsular esto en una función porque lo vamos a utilizar más veces

In [27]:
def get_consecutive_tokens_with_max_frequency(dict_tokens_by_word_appearance, list_corpus_tokens):
    dict_keys = dict_tokens_by_word_appearance.keys()

    list_consecutive_tokens = []
    for i, key in enumerate(dict_keys):
        # Get the tokens of the word
        number_of_toneks_of_word = len(dict_tokens_by_word_appearance[key]["tokens"])

        # Get consecituve tokens
        for j in range(number_of_toneks_of_word-1):
            # Get consecutive tokens
            consecutive_tokens = dict_tokens_by_word_appearance[key]["tokens"][j] + dict_tokens_by_word_appearance[key]["tokens"][j+1]
            # Append the consecutive tokens to the list
            for _ in range(dict_tokens_by_word_appearance[key]["count"]):
                list_consecutive_tokens.append(consecutive_tokens)

    # Get consecutive tokens with maximum frequency
    dict_consecutive_tokens = {}
    for token in list_consecutive_tokens:
        # Check if the token is already in the dictionary
        if token in dict_consecutive_tokens:
            # Increment the count of the token
            dict_consecutive_tokens[token] += 1
        
        # If the token is not in the dictionary
        else:
            # Add the token to the dictionary
            dict_consecutive_tokens[token] = 1

    # Get the consecutive token with maximum frequency
    max_consecutive_token = None
    while True:
        # Get the token with maximum frequency
        consecutive_token = max(dict_consecutive_tokens, key=dict_consecutive_tokens.get)

        # Check if the token is already in the list of tokens
        if consecutive_token in list_corpus_tokens:
            # Remove token from the dictionary
            dict_consecutive_tokens.pop(consecutive_token)

        # If the token is not in the list of tokens
        else:
            # Assign the token to the max_consecutive_token
            max_consecutive_token = consecutive_token
            break

    return max_consecutive_token

Comprobamos que obtenemos lo mismo que antes

In [28]:
max_consecutive_token = get_consecutive_tokens_with_max_frequency(dict_tokens_by_word_appearance, initial_corpus_tokens)
print(f"Consecutive token with maximum frequency: {max_consecutive_token}")

Consecutive token with maximum frequency: ug


Vemos que sí

Ahora nuestro corpus de tokens se puede modificar añadiendo el token `ug`

In [29]:
# new_corpus_tokens = initial_corpus_tokens + max_consecutive_token
new_corpus_tokens = initial_corpus_tokens.copy()
new_corpus_tokens.add(max_consecutive_token)

print(f"Initial corpus tokens: {initial_corpus_tokens}")
print(f"New corpus tokens: {new_corpus_tokens}")

Initial corpus tokens: {'p', 'n', 'u', 's', 'h', 'g', 'b'}
New corpus tokens: {'p', 'n', 'ug', 'g', 'b', 'u', 's', 'h'}


Metemos esto también en una función

In [30]:
def get_new_corpus_tokens(max_consecutive_token, initial_corpus_tokens):
    new_corpus_tokens = initial_corpus_tokens.copy()
    new_corpus_tokens.add(max_consecutive_token)
    return new_corpus_tokens

Volveremos a comprobar que obtenemos lo mismo que antes

In [31]:
new_corpus_tokens = get_new_corpus_tokens(max_consecutive_token, initial_corpus_tokens)
print(f"Initial corpus tokens: {initial_corpus_tokens}")
print(f"New corpus tokens: {new_corpus_tokens}")

Initial corpus tokens: {'p', 'n', 'u', 's', 'h', 'g', 'b'}
New corpus tokens: {'p', 'n', 'ug', 'g', 'b', 'u', 's', 'h'}


Vemos que sí

Ahora vamos a modificar el diccionario en el que aparecen las palabras, los tokens y el número de veces que aparecen con el nuevo token

In [32]:
import copy

dict_keys = dict_tokens_by_word_appearance.keys()
dict_tokens_by_word_appearance_tmp = copy.deepcopy(dict_tokens_by_word_appearance)

for key in dict_keys:
    # Check if the new token is in the word
    if max_consecutive_token in key:
        print(f"Token {max_consecutive_token} is in the word {key}")

        # Add the new token to the word tokens
        dict_tokens_by_word_appearance_tmp[key]["tokens"].append(max_consecutive_token)

        print(f"New tokens of the word {key}: {dict_tokens_by_word_appearance_tmp[key]['tokens']}")

print(f"Initial tokens by word appearance: {dict_tokens_by_word_appearance}")
print(f"New tokens by word appearance: ")
dict_tokens_by_word_appearance_tmp

Token ug is in the word hug
New tokens of the word hug: ['h', 'u', 'g', 'ug']
Token ug is in the word pug
New tokens of the word pug: ['p', 'u', 'g', 'ug']
Token ug is in the word hugs
New tokens of the word hugs: ['h', 'u', 'g', 's', 'ug']
Initial tokens by word appearance: {'hug': {'count': 10, 'tokens': ['h', 'u', 'g']}, 'pug': {'count': 5, 'tokens': ['p', 'u', 'g']}, 'pun': {'count': 12, 'tokens': ['p', 'u', 'n']}, 'bun': {'count': 4, 'tokens': ['b', 'u', 'n']}, 'hugs': {'count': 5, 'tokens': ['h', 'u', 'g', 's']}}
New tokens by word appearance: 


{'hug': {'count': 10, 'tokens': ['h', 'u', 'g', 'ug']},
 'pug': {'count': 5, 'tokens': ['p', 'u', 'g', 'ug']},
 'pun': {'count': 12, 'tokens': ['p', 'u', 'n']},
 'bun': {'count': 4, 'tokens': ['b', 'u', 'n']},
 'hugs': {'count': 5, 'tokens': ['h', 'u', 'g', 's', 'ug']}}

Metemos esto en una función

In [33]:
def update_tokens_by_word_appearance(dict_tokens_by_word_appearance, max_consecutive_token):
    dict_tokens_by_word_appearance_tmp = copy.deepcopy(dict_tokens_by_word_appearance)
    dict_keys = dict_tokens_by_word_appearance_tmp.keys()

    for key in dict_keys:
        # Check if the new token is in the word
        if max_consecutive_token in key:
            # Add the new token to the word tokens
            dict_tokens_by_word_appearance_tmp[key]["tokens"].append(max_consecutive_token)

    return dict_tokens_by_word_appearance_tmp

Comprobamos que está bien

In [34]:
dict_tokens_by_word_appearance = update_tokens_by_word_appearance(dict_tokens_by_word_appearance, max_consecutive_token)
print(f"New tokens by word appearance: ")
dict_tokens_by_word_appearance

New tokens by word appearance: 


{'hug': {'count': 10, 'tokens': ['h', 'u', 'g', 'ug']},
 'pug': {'count': 5, 'tokens': ['p', 'u', 'g', 'ug']},
 'pun': {'count': 12, 'tokens': ['p', 'u', 'n']},
 'bun': {'count': 4, 'tokens': ['b', 'u', 'n']},
 'hugs': {'count': 5, 'tokens': ['h', 'u', 'g', 's', 'ug']}}

En resumen, en una primera iteracción hemos pasado de un corpus de tokens `s, g, h, u, n, p, b` al nuevo corpus de tokens `h, u, n, p, s, g, b, ug`

Realizamos ahora una segunda iteración, obtenemos el par de tokens consecutivos que más veces aparecen en el diccionario

In [35]:
max_consecutive_token = get_consecutive_tokens_with_max_frequency(dict_tokens_by_word_appearance, new_corpus_tokens)
print(f"Consecutive token with maximum frequency: {max_consecutive_token}")

Consecutive token with maximum frequency: pu


Obtenemos el nuevo corpus de tokens

In [36]:
corpus_tokens = get_new_corpus_tokens(max_consecutive_token, new_corpus_tokens)
print(f"Initial corpus tokens: {new_corpus_tokens}")
print(f"New corpus tokens: {corpus_tokens}")

Initial corpus tokens: {'p', 'n', 'ug', 'g', 'b', 'u', 's', 'h'}
New corpus tokens: {'p', 'n', 'pu', 'u', 's', 'h', 'ug', 'g', 'b'}


Y obtenemos el nuevo diccionario en el que aparecen las palabras, los tokens y el número de veces que aparecen

In [37]:
dict_tokens_by_word_appearance = update_tokens_by_word_appearance(dict_tokens_by_word_appearance, max_consecutive_token)
print(f"New tokens by word appearance: ")
dict_tokens_by_word_appearance

New tokens by word appearance: 


{'hug': {'count': 10, 'tokens': ['h', 'u', 'g', 'ug']},
 'pug': {'count': 5, 'tokens': ['p', 'u', 'g', 'ug', 'pu']},
 'pun': {'count': 12, 'tokens': ['p', 'u', 'n', 'pu']},
 'bun': {'count': 4, 'tokens': ['b', 'u', 'n']},
 'hugs': {'count': 5, 'tokens': ['h', 'u', 'g', 's', 'ug']}}

Ahora podemos seguir hasta tener un corpus de tokens con el tamaño que queramos, vamos a crear un corpus de 15 tokens

In [38]:
len_corpus_tokens = 15

while len(corpus_tokens) < len_corpus_tokens:
    max_consecutive_token = get_consecutive_tokens_with_max_frequency(dict_tokens_by_word_appearance, corpus_tokens)
    print(f"Consecutive token with maximum frequency: {max_consecutive_token}")

    # If there are no more consecutive tokens break the loop
    if max_consecutive_token is None:
        break

    corpus_tokens = get_new_corpus_tokens(max_consecutive_token, corpus_tokens)
    print(f"New corpus tokens: {corpus_tokens}")

    dict_tokens_by_word_appearance = update_tokens_by_word_appearance(dict_tokens_by_word_appearance, max_consecutive_token)
    print(f"New tokens by word appearance: {dict_tokens_by_word_appearance}\n")

Consecutive token with maximum frequency: un
New corpus tokens: {'p', 'n', 'pu', 'un', 'u', 's', 'h', 'ug', 'g', 'b'}
New tokens by word appearance: {'hug': {'count': 10, 'tokens': ['h', 'u', 'g', 'ug']}, 'pug': {'count': 5, 'tokens': ['p', 'u', 'g', 'ug', 'pu']}, 'pun': {'count': 12, 'tokens': ['p', 'u', 'n', 'pu', 'un']}, 'bun': {'count': 4, 'tokens': ['b', 'u', 'n', 'un']}, 'hugs': {'count': 5, 'tokens': ['h', 'u', 'g', 's', 'ug']}}

Consecutive token with maximum frequency: hu
New corpus tokens: {'p', 'hu', 'n', 'pu', 'un', 'u', 's', 'h', 'ug', 'g', 'b'}
New tokens by word appearance: {'hug': {'count': 10, 'tokens': ['h', 'u', 'g', 'ug', 'hu']}, 'pug': {'count': 5, 'tokens': ['p', 'u', 'g', 'ug', 'pu']}, 'pun': {'count': 12, 'tokens': ['p', 'u', 'n', 'pu', 'un']}, 'bun': {'count': 4, 'tokens': ['b', 'u', 'n', 'un']}, 'hugs': {'count': 5, 'tokens': ['h', 'u', 'g', 's', 'ug', 'hu']}}

Consecutive token with maximum frequency: gug
New corpus tokens: {'p', 'hu', 'n', 'pu', 'un', 'gug',

Ahora que hemos visto cómo se entrena el tokenizador BPE, vamos a entrenarlo desde cero para afianzar los conocimientos

In [46]:
corpus_words = ["hug", "pug", "pun", "bun", "hugs"]

# Concatenate all the words in the corpus
initial_corpus_tokens = ""
for word in corpus_words:
    initial_corpus_tokens += word

# Convert the concatenated string to a set of tokens to get unique tokens
corpus_tokens = set(initial_corpus_tokens)

print(f"Corpus words: {corpus_words}")
print(f"Initial corpus tokens: {corpus_tokens}")
print(f"Number of initial corpus tokens: {len(corpus_tokens)}")

Corpus words: ['hug', 'pug', 'pun', 'bun', 'hugs']
Initial corpus tokens: {'p', 'n', 'u', 's', 'h', 'g', 'b'}
Number of initial corpus tokens: 7


In [47]:
corpus = [
    "hug hug hug pun pun bun hugs",
    "hug hug pug pug pun pun hugs",
    "hug hug pug pug pun pun pun pun hugs",
    "pug pun pun pun bun hugs",
    "hug hug hug pun bun bun hugs",
]

num_hug = 0
num_pug = 0
num_pun = 0
num_bun = 0
num_hugs = 0

for sentence in corpus:
    words = sentence.split(" ")
    for word in words:
        if word == "hug":
            num_hug += 1
        elif word == "pug":
            num_pug += 1
        elif word == "pun":
            num_pun += 1
        elif word == "bun":
            num_bun += 1
        elif word == "hugs":
            num_hugs += 1

dict_tokens_by_word_appearance = {
    "hug":
        {
            "count": num_hug,
            "tokens": [character for character in "hug"],
        },
    "pug":
        {
            "count": num_pug,
            "tokens": [character for character in "pug"],
        },
    "pun":
        {
            "count": num_pun,
            "tokens": [character for character in "pun"],
        },
    "bun":
        {
            "count": num_bun,
            "tokens": [character for character in "bun"],
        },
    "hugs":
        {
            "count": num_hugs,
            "tokens": [character for character in "hugs"],
        },
}

dict_tokens_by_word_appearance

{'hug': {'count': 10, 'tokens': ['h', 'u', 'g']},
 'pug': {'count': 5, 'tokens': ['p', 'u', 'g']},
 'pun': {'count': 12, 'tokens': ['p', 'u', 'n']},
 'bun': {'count': 4, 'tokens': ['b', 'u', 'n']},
 'hugs': {'count': 5, 'tokens': ['h', 'u', 'g', 's']}}

Lo entrenamos desde cero hasta obtener un corpus de 15 tokens

In [48]:
len_corpus_tokens = 15
print(f"Initial corpus tokens: ({len(corpus_tokens)}) {corpus_tokens}")

while len(corpus_tokens) < len_corpus_tokens:
    max_consecutive_token = get_consecutive_tokens_with_max_frequency(dict_tokens_by_word_appearance, corpus_tokens)

    # If there are no more consecutive tokens break the loop
    if max_consecutive_token is None:
        break

    corpus_tokens = get_new_corpus_tokens(max_consecutive_token, corpus_tokens)

    dict_tokens_by_word_appearance = update_tokens_by_word_appearance(dict_tokens_by_word_appearance, max_consecutive_token)

print(f"New corpus tokens: ({len(corpus_tokens)}) {corpus_tokens}")

Initial corpus tokens: (7) {'p', 'n', 'u', 's', 'h', 'g', 'b'}
New corpus tokens: (15) {'p', 'hu', 'n', 'npu', 'pu', 'un', 'gug', 'puun', 'u', 's', 'h', 'ughu', 'ug', 'g', 'b'}


## Tokenización

Si ahora quisiéramos tokenizar, primero tendríamos que crear un vocabulario, es decir, asignar a cada token un ID

In [54]:
vocab = {}
for i, token in enumerate(corpus_tokens):
    vocab[token] = i

print(f"Vocabulary: ")
vocab

Vocabulary: 


{'p': 0,
 'hu': 1,
 'sug': 2,
 'npu': 3,
 'ugpu': 4,
 'gug': 5,
 'u': 6,
 'ug': 7,
 'ughu': 8,
 'n': 9,
 'pu': 10,
 'un': 11,
 'puun': 12,
 's': 13,
 'h': 14,
 'gs': 15,
 'g': 16,
 'b': 17}

Lo metemos en una función

In [55]:
def get_vocabulary(corpus_tokens):
    vocab = {}
    for i, token in enumerate(corpus_tokens):
        vocab[token] = i
    return vocab

Comprobamos que está bien

In [56]:
vocab = get_vocabulary(corpus_tokens)
print(f"Vocabulary: ")
vocab

Vocabulary: 


{'p': 0,
 'hu': 1,
 'sug': 2,
 'npu': 3,
 'ugpu': 4,
 'gug': 5,
 'u': 6,
 'ug': 7,
 'ughu': 8,
 'n': 9,
 'pu': 10,
 'un': 11,
 'puun': 12,
 's': 13,
 'h': 14,
 'gs': 15,
 'g': 16,
 'b': 17}

Si ahora queremos tokenizar la palabra `bug` podemos hacer

In [63]:
word = 'bug'

# Get the maximum length of tokens
max_len = max(len(token) for token in vocab)
print(f"Maximum length of tokens: {max_len}")

# Create a empty list of tokens
tokens = []
while len(word) > 0:
    # Flag to check if the token is found
    found = False

    # Iterate over the maximum length of tokens from max_len to 0
    for i in range(max_len, 0, -1):
        # Get the prefix of the word
        prefix = word[:i]
        print(f"Prefix: {prefix}")

        # Check if the prefix is in the vocabulary
        if prefix in vocab:
            print(f"prefix {prefix} is in the vocabulary")
            tokens.append(prefix)
            word = word[i:]
            found = True
            break
    
    # if not found:
    #     tokens.append('<UNK>')
    #     word = word[1:]

print(f"Tokens: {tokens}")

Maximum length of tokens: 4
Prefix: bug
Prefix: bug
Prefix: bu
Prefix: b
prefix b is in the vocabulary
Prefix: ug
prefix ug is in the vocabulary
Tokens: ['b', 'ug']


Pero si ahora queremos tokenizar la palabra `mug` no podríamos porque el caracter `m` no está en el vocabulario, para ello lo tokenizamos con el token `<UNK>`

In [70]:
word = 'mug'

# Get the maximum length of tokens
max_len = max(len(token) for token in vocab)
print(f"Maximum length of tokens: {max_len}")

# Create a empty list of tokens
tokens = []
while len(word) > 0:
    # Flag to check if the token is found
    found = False

    # Iterate over the maximum length of tokens from max_len to 0
    for i in range(max_len, 0, -1):
        # Get the prefix of the word
        prefix = word[:i]
        print(f"Prefix: {prefix}")

        # Check if the prefix is in the vocabulary
        if prefix in vocab:
            print(f"prefix {prefix} is in the vocabulary")
            tokens.append(prefix)
            word = word[i:]
            found = True
            break

    if not found:
        tokens.append('<UNK>')
        word = word[1:]

print(f"Tokens: {tokens}")

Maximum length of tokens: 4
Prefix: mug
Prefix: mug
Prefix: mu
Prefix: m
Prefix: ug
prefix ug is in the vocabulary
Tokens: ['<UNK>', 'ug']


Lo metemos en una función

In [71]:
def tokenize_word(word, vocab):
    # Get the maximum length of tokens
    max_len = max(len(token) for token in vocab)

    # Create a empty list of tokens
    tokens = []
    while len(word) > 0:
        # Flag to check if the token is found
        found = False

        # Iterate over the maximum length of tokens from max_len to 0
        for i in range(max_len, 0, -1):
            # Get the prefix of the word
            prefix = word[:i]

            # Check if the prefix is in the vocabulary
            if prefix in vocab:
                tokens.append(prefix)
                word = word[i:]
                found = True
                break

        if not found:
            tokens.append('<UNK>')
            word = word[1:]

    return tokens

Comprobamos que está bien

In [72]:
print(f"Tokenization of the word 'bug': {tokenize_word('bug', vocab)}")
print(f"Tokenization of the word 'mug': {tokenize_word('mug', vocab)}")

Tokenization of the word 'bug': ['b', 'ug']
Tokenization of the word 'mug': ['<UNK>', 'ug']


## Visualizador de tokens

Ahora que sabemos cómo funciona un tokenizador BPE, vamos a ver mediante el visualizador [the-tokenizer-playground](https://huggingface.co/spaces/Xenova/the-tokenizer-playground) cómo quedarían los tokens de cualquier sentencia

<iframe>
	src="https://xenova-the-tokenizer-playground.static.hf.space"
	frameborder="0"
	width="850"
	height="450"
></iframe>